In [3]:
import os
import numpy as np
import tensorflow as tf
from data_loader import DataGenerator
from sklearn.metrics import f1_score, precision_score, recall_score
import datetime
from helpers import f1 as f1_metric
from model import lstm_resnet

def set_random_seeds(seed):
    """Set random seeds for reproducibility."""
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

def create_data_generators():
    """Create data generators for training and validation."""
    train_gen = DataGenerator(
        image_dir="../Unet/data_v3_processed/train/images/images",
        mask_dir="../Unet/data_v3_processed/train/masks/masks",
        batch_size=8,
        time_steps=15,
        img_size=(256,256),
        normalize_images=True,
        normalize_masks=False,
        infinite=True,
    )
        
    val_gen = DataGenerator(
        image_dir="../Unet/data_v3_processed/val/images/images",
        mask_dir="../Unet/data_v3_processed/val/masks/masks",
        batch_size=8,
        time_steps=15,
        img_size=(256,256),
        normalize_images=True,
        normalize_masks=False,
        infinite=True
    )

    return train_gen, val_gen

def train_and_evaluate_model(run_number, train_generator, val_generator, 
                           epochs=4, steps_per_epoch=1000, validation_steps=5, 
                           random_seed=None):
    """Train a single model and return evaluation metrics."""
    print(f"\nTraining Run {run_number + 1}/5 (Seed: {random_seed})")
    print("=" * 50)
    
    if random_seed is not None:
        set_random_seeds(random_seed)
    
    model = lstm_resnet(256, 256, 3, 15)
    
    model.compile(optimizer='adam', 
                 loss='binary_crossentropy', 
                 metrics=[f1_metric])
    
    try:
        history = model.fit(train_generator,
                           validation_data=val_generator,
                           epochs=epochs,
                           steps_per_epoch=steps_per_epoch,
                           validation_steps=validation_steps,
                           verbose=1)

        model_path = os.path.join("models", f"lstm_resnet_run{run_number+1}.h5")
        os.makedirs("models", exist_ok=True)
        model.save(model_path)
        
        return model_path
        
    except Exception as e:
        print(f"ERROR in training: {e}")
        return None

# Main execution
random_seeds = [42, 123]

for run in range(2):
    # CREATE FRESH GENERATORS FOR EACH RUN
    train_generator, val_generator = create_data_generators()
    
    model_path = train_and_evaluate_model(
        run, train_generator, val_generator, random_seed=random_seeds[run]
    )
    
    tf.keras.backend.clear_session()


Training Run 1/5 (Seed: 42)
Epoch 1/4
1000/1000 [==============================] - 1298s 1s/step - loss: 0.0225 - f1: 0.7107 - val_loss: 0.0066 - val_f1: 0.7393
Epoch 2/4
1000/1000 [==============================] - 1293s 1s/step - loss: 0.0017 - f1: 0.9358 - val_loss: 0.0134 - val_f1: 0.7245
Epoch 3/4
1000/1000 [==============================] - 1292s 1s/step - loss: 9.8689e-04 - f1: 0.9556 - val_loss: 0.0110 - val_f1: 0.6963
Epoch 4/4
1000/1000 [==============================] - 1291s 1s/step - loss: 0.0015 - f1: 0.9289 - val_loss: 0.0150 - val_f1: 0.7096

Training Run 2/5 (Seed: 123)
Epoch 1/4
1000/1000 [==============================] - 1298s 1s/step - loss: 0.0144 - f1: 0.7491 - val_loss: 0.0109 - val_f1: 0.6603
Epoch 2/4
1000/1000 [==============================] - 1292s 1s/step - loss: 0.0014 - f1: 0.9444 - val_loss: 0.0114 - val_f1: 0.7571
Epoch 3/4
1000/1000 [==============================] - 1292s 1s/step - loss: 7.7222e-04 - f1: 0.9643 - val_loss: 0.0125 - val_f1: 0.7482
Ep

In [4]:
import shutil

shutil.make_archive('models', 'zip', 'models')

'/home/y2b/cnn_lstm_root_architecture/LSTM-ResNet/models.zip'